In [1]:
## Use DASH to build user GUI
# %pip install dash
# %pip install nbformat

from dash import Dash, html, dcc, callback, Output, Input, ctx
import plotly.express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from presets import setmedium
import calculateRayleighIntegral
import settransducer


In [2]:
app = Dash('tempCompliance4HIFU')

df_pressure2D = pd.read_csv(r"C:\Users\geral\OneDrive\Documents\GitHub\TempCompliance4HIFU\src\matlab_data_1MHz_2cm_5cm.csv")  # sample_data
df_pressure2D = np.rot90(df_pressure2D)
z_axis = np.linspace(0,0.01,100)
r_axis = np.linspace(-0.02,0.02,101)

app.layout = [

    # Title
    html.H1(children='Temperature Compliance for HIFU Transducers',style={'textAlign':'center'}), 


    # TOP Section - Button and Messages
    html.Div([
        html.Button('CALCULATE PRESSURE AND TEMPERATURE',id='button',n_clicks=0),
        html.Div(id='message_Display',style={'textAlign':'left'}),
    ],style={'width': '50%','float': 'right','display': 'inline-block','padding': '10px 5px'}),
    
    
    # RIGHT Section - DISPLAYS
    html.Div([
        html.H2('Results', style ={'textAlign':'center'}), 
            dcc.Dropdown(['Pressure','Intensity'],'Intensity',id='DROP_field2D'),
            dcc.Graph(id='GRAPH_field2D',hoverData={'points': [{'customdata': 'Pressure'}]}),
            dcc.Graph(id='GRAPH_time1D'),
    ],style={'width': '50%', 'float': 'right', 'display': 'inline-block'}),


    # SPACE
    html.Div([html.H3(' ',style ={'textAlign':'center'})],style={'width': '3%', 'float': 'right', 'display': 'inline-block'}),


    # MIDDLE Section - INPUTS
    html.Div([     
            
        # Trandsucer Section (Top)
        html.H3('Transducer', style ={'textAlign':'center'}), 
        html.P('Frequency [MHz]', style ={'textAlign':'left'}),
            dcc.Input(id="Frequency", type='number', placeholder='Input Frequency',value=1e6),
        html.P('Radius [mm]', style ={'textAlign':'left'}),
            dcc.Input(id="Radius", type='number', placeholder='Input Radius',value=0.02),
        html.P('Focus Distance [mm]', style ={'textAlign':'left'}),
            dcc.Input(id="Focus", type='number', placeholder='Input Focus Distance',value=0.05),
        html.P('Initial Pressure [mm]', style ={'textAlign':'left'}),
            dcc.Input(id="InitPress", type='number', placeholder='Input Initial Pressure',value=1),


        # Heating Section (Middle)
        html.H3('Heating', style ={'textAlign':'center'}), 
        html.P('Time Heating [s]', style ={'textAlign':'left'}),
            dcc.Input(id="HeatTime", type='number', placeholder='Input Heat Time'),
        html.P('Time Cooling [s]', style ={'textAlign':'left'}),
            dcc.Input(id="CoolTime", type='number', placeholder='Input Cool Time'),
        html.P('Duty Cycle [Percentage]', style ={'textAlign':'left'}),
            dcc.Input(id="DutyCycle", type='number', placeholder='Input Duty Cycle'),


        # Plotting Section
        html.H3('Plotting Parameters', style ={'textAlign':'center'}), 
        html.P('Number of Axial Steps', style ={'textAlign':'left'}),
            dcc.Input(id="numZ", type='number', placeholder='Input # Axial Steps',value=100),
        html.P('Number of Radial Steps', style ={'textAlign':'left'}),
            dcc.Input(id="numR", type='number', placeholder='Input # Radial Steps',value=100),
        html.P('Time Step', style ={'textAlign':'left'}),
            dcc.Input(id="numTime", type='number', placeholder='Input Time Step'),

    ], style={'width': '15%', 'float': 'right', 'display': 'inline-block'}),


    # SPACE
    html.Div([html.H3(' ',style ={'textAlign':'center'})],style={'width': '3%', 'float': 'right', 'display': 'inline-block'}),


    # LEFT Section - Medium 
    html.Div([  
        html.H3('Medium', style ={'textAlign':'center'}), 
        html.P('Presets', style ={'textAlign':'left'}), 
            dcc.Dropdown(['Custom','Water','Glycerol','Egg White','Castor Oil'],'Water',id='DROP_medium'),
        html.P('Speed of Sound [m/s]', style ={'textAlign':'left'}), 
            dcc.Input(id="Speed", type='number', placeholder='Input Speed of Sound'),
        html.P('Density [kg/m^3]', style ={'textAlign':'left'}),
            dcc.Input(id="Density", type='number', placeholder='Input Density'),
        html.P('Absorption Coeffient [Np/(m*MHz^2)]', style ={'textAlign':'left'}),
            dcc.Input(id="AbsCoeff", type='number', placeholder='Input Absorption Coefficient'),
        html.P('Specific Heat Capacity [J/(kg*K)]', style ={'textAlign':'left'}),
            dcc.Input(id="SpecHeatCap", type='number', placeholder='Input Specific Heat Capacity'),
        html.P('Thermal Diffusivity [(m^2)/s]', style ={'textAlign':'left'}),
            dcc.Input(id="ThermDiff", type='number', placeholder='Input Thermal Diffusivity'),  
    ], style={'width': '15%', 'float': 'right', 'display': 'inline-block'}),

]

##### ##### Callbacks ##### ##### 


# Update 2D Figure (Pressure or Intensity)
@callback(
    Output('GRAPH_field2D', 'figure'),
    Input('DROP_field2D','value')
)
def update2Dfigure(DROP_field2D):
    # Update the 2D Figure Depending on Pressure or Intensity Selection
    match DROP_field2D:
        case 'Intensity':
            display_array = pow(df_pressure2D,2)
        case 'Pressure':
            display_array = df_pressure2D
    # MatPlotLib Figure Structure
    fig2D = px.imshow(
        display_array, x=z_axis, y=r_axis,
        aspect='auto', color_continuous_scale='jet', 
    )
    return fig2D


# Set Medium Properties Callback
@callback(
    Output('Speed','value'),
    Output('Density','value'),
    Output('AbsCoeff','value'),
    Output('SpecHeatCap','value'),
    Output('ThermDiff','value'),
    Input('DROP_medium','value'),
    Input('Speed','value'),
    Input('Density','value'),
    Input('AbsCoeff','value'),
    Input('SpecHeatCap','value'),
    Input('ThermDiff','value'),
)
def getMedium(DROP_medium, Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff):
    mediumUserIn = collectMediumUserInputs(Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff)
    mediumProp = setmedium.setMedium(DROP_medium, mediumUserIn)
    return mediumProp['speed'], mediumProp['density'], mediumProp['absCoeff'], mediumProp['specHeatCap'], mediumProp['thermDiff']

def collectMediumUserInputs(Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff):
    if Speed == None: Speed = 0
    if Density == None: Density = 0
    if AbsCoeff == None: AbsCoeff = 0
    if SpecHeatCap == None: SpecHeatCap = 0
    if ThermDiff == None: ThermDiff = 0
    userInputs = dict(speed = Speed, density = Density, absCoeff = AbsCoeff, specHeatCap = SpecHeatCap, thermDiff = ThermDiff)
    return userInputs


# Calculate Pressure and Temperature Field (When Button is Pressed)
@callback(
    Output('message_Display', 'children'),
    
    Input('button','n_clicks'),

    Input('Frequency','value'),
    Input('Radius','value'),
    Input('Focus','value'),
    Input('InitPress','value'),

    Input('DROP_medium','value'),
    Input('Speed','value'),
    Input('Density','value'),
    Input('AbsCoeff','value'),
    Input('SpecHeatCap','value'),
    Input('ThermDiff','value'),

    Input('numZ','value'),
    Input('numR','value'),

    # Input('HeatTime','value'),
    # Input('CoolTime','value'),
    # Input('DutyCycle','value'),

    Input('DROP_field2D','value'),

)
def calculate2DField(buttonClicks,Frequency, Radius, Focus, InitPress, 
                     DROP_medium, Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff,
                     numZ, numR, 
                     DROP_field2D):
    # When the Button is Pressed
    if 'button' == ctx.triggered_id:
        msg = "Calculating... [Estimated Time Completion 3.5 Minutes]"
        
        # Set Input Parameters
        trans = settransducer.setTransducer(Frequency,Radius,Focus,InitPress) # freq, radius, focus, init pressure
        mediumUserIn = collectMediumUserInputs(Speed, Density, AbsCoeff, SpecHeatCap, ThermDiff)
        medium = setmedium.setMedium(DROP_medium, mediumUserIn)
        field = dict(numAxialStep = numZ, numRadialStep = numR)

        # Rayleigh Integral
        df_pressure2D, z_axis, r_axis, iscomplete = calculateRayleighIntegral.generateField(trans,medium,field)
        if iscomplete == 0:
            msg = "ERROR: RAYLEIGH INTEGRAL INCOMPLETE"
            return html.Div(msg)
        
        msg = 'Complete'
        return html.Div(msg), df_pressure2D, z_axis, r_axis, 

    # When Button is Not Pressed
    else:
        if buttonClicks == 0:
            msg = "Program Loaded In Browser. Awaiting Inputs"
            return html.Div(msg)
        else:
            msg = "_"
            return html.Div(msg)






# Execute App
if __name__ == '__main__':
    app.run(debug=True)
